In [6]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from textblob import TextBlob
from scipy.sparse import csr_matrix
from nltk.corpus import words

english_corpus = set(words.words())

matrix = pd.read_csv('matrix.csv')
#matrix = matrix.iloc[:, :100]
#pd.DataFrame(list(matrix.columns), columns = ['feature']).to_csv('matrix_columns.csv', index = False)
matrix

,aa,aaa,aaaaaaaa,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa,aaaaand,aaas,aab,aabad,aabout,aaca,...,zurich,zvun,zwaar,zwkmjkkz,zxcap,zxje,zxlo,zy,zz,zzz
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [2]:
from tqdm import tqdm
import numpy as np

from textblob import Word

def spellcheck(word, threshold):
    value = Word(word).spellcheck()[0]
    if (value[1] < threshold):
        return f'-1'
    return value[0]


def correct(text):
    return TextBlob(text).correct().string

np_spellcheck = np.vectorize(spellcheck)

def process_words(columns):
    word_list = [(word, spellcheck(word, 0.7)) for word in columns]
    word_list = dict(word_list)
    return word_list
    


In [7]:
columns_df = pd.DataFrame(list(matrix.columns), columns = ['feature'])
columns_df

,feature
0,aa
1,aaa
2,aaaaaaaa
3,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa...
4,aaaaand
...,...
29223,zxje
29224,zxlo
29225,zy
29226,zz


In [8]:
tqdm.pandas(desc="feature_spellcheck")
%time columns_df['feature_spellcheck'] = columns_df['feature'].progress_apply(lambda word: spellcheck(word, 0.7))
columns_df

feature_spellcheck: 100%|██████████| 29228/29228 [1:04:35<00:00,  7.54it/s]

CPU times: user 1h 2min 59s, sys: 21.5 s, total: 1h 3min 20s
Wall time: 1h 4min 35s


,feature,feature_spellcheck
0,aa,-1
1,aaa,-1
2,aaaaaaaa,-1
3,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa...,-1
4,aaaaand,-1
...,...,...
29223,zxje,-1
29224,zxlo,-1
29225,zy,by
29226,zz,-1


In [9]:
columns_df.to_csv('matrix_features.csv', index = True)

In [5]:
np_features = np.array(list(matrix.columns))
np_spellcheck(np_features)

TypeError: spellcheck() missing 1 required positional argument: 'threshold'

In [ ]:
word_list = [
    'aa',
    'aaa',
    'aaaaaaaa',
    'aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa',
    'aaaaand',
    'aaas',
    'aab',
    'aabad',
    'aabout',
    'aaca',
    'aacf',
    'aadtth',
    'aah',
    'aahar',
    'aai',
    'aaj',
    'aalst',
    'aap',
    'aar',
    'aaron']

word_list = [(word, spellcheck(word, 0.7)) for word in list(matrix.columns)]
word_list = dict(word_list)
word_list

In [ ]:
values = set(word_list.values())
values = list(values.intersection(english_corpus))
values

In [ ]:
word_list = dict([(key, word_list[key] if word_list[key] in values else -1) for key in word_list.keys()])
word_list

In [ ]:
drop_columns = [key for key in word_list.keys() if word_list[key] == -1]
drop_columns

In [ ]:
rename_columns = dict([(key, word_list[key]) for key in word_list.keys() if word_list[key] != -1])
rename_columns

In [ ]:
matrix = matrix.drop(drop_columns, axis = 1)
matrix

In [ ]:
matrix.rename(columns=rename_columns, inplace=True)
matrix

In [ ]:

spellcheck('aaaaaaaaaaaa', 0.6)

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm

df = pd.DataFrame(np.random.randint(0, 100, (100000, 6)))

# Register `pandas.progress_apply` and `pandas.Series.map_apply` with `tqdm`
# (can use `tqdm.gui.tqdm`, `tqdm.notebook.tqdm`, optional kwargs, etc.)
tqdm.pandas(desc="my bar!")

# Now you can use `progress_apply` instead of `apply`
# and `progress_map` instead of `map`
df.progress_apply(lambda x: x**2)